# Install any required library

In [3]:
%pip install graphviz
%pip install pandas
%pip install rdflib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Import any required data
Required data including _pandas_, _rdflib_ and its classes like Graph, Namespace, Literal etc.

In [17]:
import locale # For currency formatting

import pandas as pd
from graphviz import Digraph
from rdflib import Graph, Namespace, RDF, Literal, XSD
from rdflib.plugins.sparql import prepareQuery

## Import dataset
Importing the dataset which in this case is **NBA2k2x** data player

In [5]:
df = pd.read_csv('../docs/nba2k_data.csv').fillna("No Data")

## Namespaces

Namespace is required in order for identification

In [6]:
# Instead of using http, I changed it to https protocol to be more secured.
# Notice that I do not use any "example.org" since it was too boring all the time, so I make a little tweak with url "nba-database.org" instead.

nba_namespace = Namespace("https://nba-database.org/")  # nba
nba_player = Namespace("https://nba-database.org/profile#")  # player_nba
nba_types_namespace = Namespace("https://nba-database.org/types#")  # types

In [7]:
# First Graph-Making
# Binding any necessary data.

g = Graph()
g.bind("nba", nba_namespace)
g.bind("nba_player", nba_player)
g.bind("types", nba_types_namespace)

## Creating RDF Format

First of all, all we need is an RDF file, but we don't have a single file yet, so I create it first with the format using turtle format.

In [8]:
# Just to beautify the salary column.
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

try:
    for i, row in df.iterrows():
        subj = nba_namespace[row['full_name'].replace(' ', '_').replace('-', '_')]

        g.add((subj, RDF.type, nba_types_namespace.Player))
        g.add((subj, nba_player.full_name, Literal(row['full_name'])))
        g.add((subj, nba_player.rating, Literal(row['rating'], datatype=XSD.integer)))
        g.add((subj, nba_player.jersey, Literal(row['jersey'])))
        g.add((subj, nba_player.team, Literal(row['team'])))
        g.add((subj, nba_player.position, Literal(row['position'])))
        if row['b_day']:
            month, day, year = row['b_day'].split('/')
            iso_date = f"19{year}-{month.zfill(2)}-{day.zfill(2)}"
            g.add((subj, nba_player.b_day, Literal(iso_date, datatype=XSD.date)))
        g.add((subj, nba_player.height, Literal(row['height'])))
        g.add((subj, nba_player.weight, Literal(row['weight'])))

        salary_str = row['salary']
        salary_value = locale.atof(salary_str.strip('$').replace(',', ''))  # Convert to float
        formatted_salary = locale.currency(salary_value, grouping=True)  # Format as US currency
        g.add((subj, nba_player.salary, Literal(formatted_salary)))

        g.add((subj, nba_player.country, Literal(row['country'])))
        g.add((subj, nba_player.draft_year, Literal(int(row['draft_year']), datatype=XSD.integer)))
        g.add((subj, nba_player.draft_round, Literal(row['draft_round'])))
        g.add((subj, nba_player.draft_peak, Literal(row['draft_peak'])))
        g.add((subj, nba_player.college, Literal(row['college'])))
        g.add((subj, nba_player.version, Literal(row['version'])))
    g.serialize('nba-data.ttl', format='turtle')  # Export to Turtle.
    g.serialize('nba-data.rdf', format='xml')  # Export to RDF/ XML.

    print("Successful written data.")
except IOError as e:
    print(e)

Successful written data.


## Creating a Visual Representation of RDF Format

In [9]:
graph = Graph()
graph.parse('nba-data.ttl', format='turtle')

dot_graph = Digraph(format='svg', comment='rdf into dot format.')

try:
    for s, p, o in graph:
        if p == RDF.type and o == nba_types_namespace.Player:
            subj = s

            player_data = {}
            for prop in [nba_player.full_name, nba_player.rating, nba_player.jersey,
                         nba_player.team, nba_player.position, nba_player.b_day,
                         nba_player.height, nba_player.weight, nba_player.salary,
                         nba_player.country, nba_player.draft_year, nba_player.draft_round,
                         nba_player.draft_peak, nba_player.college, nba_player.version]:
                prop_value = str(graph.value(s, prop))
                player_data[prop] = prop_value

                dot_graph.node(prop_value)

            full_name = player_data[nba_player.full_name]
            for prop, label in [(nba_player.rating, "hasRating"), (nba_player.jersey, "hasJersey"),
                                (nba_player.team, "currentTeam"), (nba_player.position, "hasPosition"),
                                (nba_player.b_day, "hasBirthday"), (nba_player.height, "hasHeight"),
                                (nba_player.weight, "hasWeight"), (nba_player.salary, "hasYearSalary"),
                                (nba_player.country, "isFromCountry")]:
                dot_graph.edge(full_name, player_data[prop], label=label)

    dot_graph.render('data.dot', view=True)
except IOError as e:
    print(f"Something has happened: {e}")

# Querying
This step I need to create several queries like:
- ASK Query
- DESCRIBE Query
- AGGREGATE Query
- etc

## Select Query

In [10]:
query_string = """
SELECT ?player ?full_name ?position ?team ?salary ?version
WHERE {
  ?player rdf:type types:Player;
          nba_player:full_name ?full_name ;
          nba_player:position ?position ;
          nba_player:team ?team ;
          nba_player:salary ?salary ;
          nba_player:version ?version .
}
LIMIT 25
"""

query_result = g.query(query_string)
dot_graph_selectQuery = Digraph(format='svg')

try:
    for idx, row in enumerate(query_result, start=1):
        player_data = {
            "full_name": str(row['full_name']),
            "position": str(row['position']),
            "team": str(row['team']),
            "salary": str(row['salary']),
            "version": str(row['version']),
        }

        for prop, label in [("position", "hasPosition"), ("team", "currentTeam"),
                            ("salary", "currentSalary"), ("version", "gameVersion")]:
            dot_graph_selectQuery.node(player_data[prop])
            dot_graph_selectQuery.edge(player_data["full_name"], player_data[prop], label=label)

        print(f"Index No: {idx}")
        print(
            f"Player URL Data: {row['player']} \nName: {row['full_name']} \nTeam: {row['team']} \nPosition: {row['position']}  \nAnnual Salary: {row['salary']} \nGame Version: {row['version']}")
        print("-------")

    dot_graph_selectQuery.render('data_select-query.dot', view=False)

except Exception as e:
    print(f"An error occurred: {e}")


Index No: 1
Player URL Data: https://nba-database.org/LeBron_James 
Name: LeBron James 
Team: Los Angeles Lakers 
Position: F  
Annual Salary: $37,436,858.00 
Game Version: NBA2k20
-------
Index No: 2
Player URL Data: https://nba-database.org/LeBron_James 
Name: LeBron James 
Team: Los Angeles Lakers 
Position: F  
Annual Salary: $37,436,858.00 
Game Version: NBA2k21
-------
Index No: 3
Player URL Data: https://nba-database.org/LeBron_James 
Name: LeBron James 
Team: Los Angeles Lakers 
Position: F  
Annual Salary: $37,436,858.00 
Game Version: NBA2k22
-------
Index No: 4
Player URL Data: https://nba-database.org/LeBron_James 
Name: LeBron James 
Team: Los Angeles Lakers 
Position: F  
Annual Salary: $39,219,566.00 
Game Version: NBA2k20
-------
Index No: 5
Player URL Data: https://nba-database.org/LeBron_James 
Name: LeBron James 
Team: Los Angeles Lakers 
Position: F  
Annual Salary: $39,219,566.00 
Game Version: NBA2k21
-------
Index No: 6
Player URL Data: https://nba-database.org/L

## ASK Query

In [11]:
query_str = """
ASK
WHERE {
  nba:LeBron_James nba_player:rating ?rating .
  FILTER (?rating > 80)
}
"""

dot_graph_query2 = Digraph(comment='ASK Query', format='svg')
rating_above_80 = False

for s, p, o in graph:
    if p == RDF.type and o == nba_types_namespace.Player:
        subject = s
        full_name = str(graph.value(s, nba_player.full_name))

        if full_name == 'LeBron James':
            rating = str(graph.value(s, nba_player.rating))
            dot_graph_query2.node(full_name)
            dot_graph_query2.node(rating)
            dot_graph_query2.edge(full_name, rating, label="hasRating")
            rating_above_80 = float(rating) > 80
            break

# dot_graph_query2.render('data_ask_query.dot', view=False)

print("Does LeBron James' rating is above 80? ")
if rating_above_80:
    print("Yes, his rating is above 80 For Sure, Obviously.")
    print("-------")
else:
    print("No, his rating is not above 80.")
    print("-------")

print("Player Data for those with rating above 80:")
for row in query_result:
    player_data = {
        "full_name": str(row['full_name']),
        "position": str(row['position']),
        "team": str(row['team']),
        "salary": str(row['salary']),
        "version": str(row['version']),
    }
    for prop, label in [("position", "hasPosition"), ("team", "currentTeam"),
                        ("salary", "currentSalary"), ("version", "gameVersion")]:
        dot_graph_selectQuery.node(player_data[prop])
        dot_graph_selectQuery.edge(player_data["full_name"], player_data[prop], label=label)

    print(f"Player URL Data: {row['player']}")
    print(f"Name: {row['full_name']}")
    print(f"Team: {row['team']}")
    print(f"Position: {row['position']}")
    print(f"Annual Salary: {row['salary']}")
    print(f"Game Version: {row['version']}")
    print("-------")

dot_graph_selectQuery.render('data_ask_query.dot', view=False)

Does LeBron James' rating is above 80? 
Yes, his rating is above 80 For Sure, Obviously.
-------
Player Data for those with rating above 80:
Player URL Data: https://nba-database.org/LeBron_James
Name: LeBron James
Team: Los Angeles Lakers
Position: F
Annual Salary: $37,436,858.00
Game Version: NBA2k20
-------
Player URL Data: https://nba-database.org/LeBron_James
Name: LeBron James
Team: Los Angeles Lakers
Position: F
Annual Salary: $37,436,858.00
Game Version: NBA2k21
-------
Player URL Data: https://nba-database.org/LeBron_James
Name: LeBron James
Team: Los Angeles Lakers
Position: F
Annual Salary: $37,436,858.00
Game Version: NBA2k22
-------
Player URL Data: https://nba-database.org/LeBron_James
Name: LeBron James
Team: Los Angeles Lakers
Position: F
Annual Salary: $39,219,566.00
Game Version: NBA2k20
-------
Player URL Data: https://nba-database.org/LeBron_James
Name: LeBron James
Team: Los Angeles Lakers
Position: F
Annual Salary: $39,219,566.00
Game Version: NBA2k21
-------
Play

'data_ask_query.dot.svg'

## DESCRIBE Query

In [12]:
query_str = """
DESCRIBE ?player
WHERE {
  ?player nba_player:full_name ?full_name .
  FILTER(?full_name = "LeBron James")
}
"""
query_result = g.query(query_str)

print(query_result.serialize(format='turtle').decode())

dot_graph_describeQuery = Digraph(comment='DESCRIBE Query', format='svg')

lebron_data = {}

for s, p, o in query_result:
    lebron_data.setdefault(s, {})
    lebron_data[s][p] = o

for s, p, o in graph:
    if p == RDF.type and o == nba_types_namespace.Player:
        subject = s
        full_name = lebron_data.get(subject, {}).get(nba_player.full_name)
        if full_name == 'LeBron James':
            dot_graph_describeQuery.node(full_name)
            for prop in [nba_player.rating, nba_player.jersey, nba_player.team,
                         nba_player.position, nba_player.b_day, nba_player.height,
                         nba_player.weight, nba_player.salary, nba_player.country]:
                value = lebron_data.get(subject, {}).get(prop)
                if value is not None:
                    dot_graph_describeQuery.node(value)
                    dot_graph_describeQuery.edge(full_name, value, label=f"has{prop.split(':')[-1]}")

dot_graph_describeQuery.render('data_describe_query.dot', view=False)

@prefix nba: <https://nba-database.org/> .
@prefix nba_player: <https://nba-database.org/profile#> .
@prefix types: <https://nba-database.org/types#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

nba:LeBron_James a types:Player ;
    nba_player:b_day "1984-12-30"^^xsd:date ;
    nba_player:college "No Data" ;
    nba_player:country "USA" ;
    nba_player:draft_peak "1" ;
    nba_player:draft_round "1" ;
    nba_player:draft_year 2003 ;
    nba_player:full_name "LeBron James" ;
    nba_player:height "6-9 / 2.06" ;
    nba_player:jersey "#23",
        "#6" ;
    nba_player:position "F" ;
    nba_player:rating 96,
        97 ;
    nba_player:salary "$37,436,858.00",
        "$39,219,566.00",
        "$41,180,544.00" ;
    nba_player:team "Los Angeles Lakers" ;
    nba_player:version "NBA2k20",
        "NBA2k21",
        "NBA2k22" ;
    nba_player:weight "250 lbs. / 113.4 kg." .



'data_describe_query.dot.svg'

## AGGREGATE Query

In [13]:
query_str = """
SELECT (COUNT(?college) as ?count_college)
WHERE {
  ?nba_player nba_player:college ?college .
  FILTER(?college = "Kentucky")
}
"""

dot_graph_aggregate_college = Digraph(comment='AGGREGATE Query (College)', format='svg')
player_data = {}

for s, p, o in graph:
    if p == RDF.type and o == nba_types_namespace.Player:
        college = str(graph.value(s, nba_player.college))
        if college == 'Kentucky':
            full_name = str(graph.value(s, nba_player.full_name))
            dot_graph_aggregate_college.node(full_name)
            dot_graph_aggregate_college.node(college)
            dot_graph_aggregate_college.edge(full_name, college, label="fromCollege")
            player_data[full_name] = {"college": college}

query_result = g.query(query_str)

for row in query_result:
    count_college = int(row['count_college'])
    if count_college > 0:
        college_name = "Kentucky"
        print(f"How many people come from {college_name}? There are: {count_college} people.")
        for full_name, data in player_data.items():
            if data["college"] == college_name:
                dot_graph_aggregate_college.node(full_name)
                dot_graph_aggregate_college.edge(full_name, college_name, label="fromCollege")

dot_graph_aggregate_college.render('data_aggregate_college.dot', view=False)

How many people come from Kentucky? There are: 26 people.


'data_aggregate_college.dot.svg'

In [14]:
query_str = """
SELECT (COUNT(?rating) as ?rating_count)
WHERE {
  ?nba_player nba_player:rating ?rating .
  FILTER(?rating < 80)
}
"""

dot_graph_aggregate_rating = Digraph(comment='AGGREGATE Query (Rating)', format='svg')
player_data = {}

for s, p, o in graph:
    if p == RDF.type and o == nba_types_namespace.Player:
        rating = str(graph.value(s, nba_player.rating))
        if int(rating) < 80:
            full_name = str(graph.value(s, nba_player.full_name))
            dot_graph_aggregate_rating.node(full_name)
            dot_graph_aggregate_rating.node(rating)
            dot_graph_aggregate_rating.edge(full_name, rating, label="yearDrafted")
            player_data[full_name] = {"rating": rating}

query_result = g.query(query_str)

for row in query_result:
    rating_count = int(row['rating_count'])
    if rating_count > 0:
        print(f"How many people have a rating below 80? There are: {rating_count} people.")
        for full_name, data in player_data.items():
            if int(data["rating"]) < 80:
                dot_graph_aggregate_rating.node(full_name)
                dot_graph_aggregate_rating.edge(full_name, data["rating"], label="yearDrafted")

dot_graph_aggregate_rating.render('data_aggregate_rating.dot', view=False)


How many people have a rating below 80? There are: 339 people.


'data_aggregate_rating.dot.svg'

## NESTED Query

In [15]:
query_str = """
SELECT (COUNT(?player) as ?count_players)
WHERE {
  ?player nba_player:full_name ?full_name;
          nba_player:position "F";
          nba_player:team "Golden State Warriors".
}
"""

dot_graph_federatedQuery = Digraph(comment='NESTED Query', format='svg')
player_names = []

for s, p, o in graph:
    if p == RDF.type and o == nba_types_namespace.Player:
        position = str(graph.value(s, nba_player.position))
        team = str(graph.value(s, nba_player.team))
        if position == 'F' and team == 'Golden State Warriors':
            full_name = str(graph.value(s, nba_player.full_name))
            dot_graph_federatedQuery.node(full_name)
            dot_graph_federatedQuery.node(position)
            dot_graph_federatedQuery.node(team)
            dot_graph_federatedQuery.edge(full_name, position, label="hasPosition")
            dot_graph_federatedQuery.edge(full_name, team, label="fromTeam")
            player_names.append(full_name)

# dot_graph_federatedQuery.render('data_nested-query.dot', view=False)

query_result = g.query(query_str)

index = 1
for row in query_result:
    count_players = int(row['count_players'])

    print(f"List of players from Los Angeles Lakers playing in Position 'F' are {count_players} people:")
    for player_name in player_names:
        print(f"{index}. Player Name: {player_name}")
        index += 1

List of players from Los Angeles Lakers playing in Position 'F' are 6 people:
1. Player Name: Glenn Robinson III
2. Player Name: Kevon Looney
3. Player Name: Alen Smailagic
4. Player Name: Eric Paschall
5. Player Name: Andrew Wiggins
6. Player Name: Draymond Green


## FEDERATED QUERY

In [22]:
from rdflib import ConjunctiveGraph

cg = ConjunctiveGraph()
cg.add((nba_player.Player, nba_player.full_name, Literal("James Harden")))

query_str = """
PREFIX nba_player: <https://nba-database.org/profile#>

SELECT ?player ?full_name ?rating ?salary ?team ?position
WHERE {
  SERVICE <https://nba-players.org/profile/james_harden> {
    ?player a nba_player:Player ;
            nba_player:full_name ?full_name ;
            nba_player:rating ?rating ;
            nba_player:salary ?salary ;
            nba_player:team ?team ;
            nba_player:position ?position.
  }
}
"""

query_result = g.query(query_str)

for s, p, o in graph:
    if p == RDF.type and o == nba_types_namespace.Player:
        team = str(graph.value(s, nba_player.team))
        rating = str(graph.value(s, nba_player.rating))
        salary = graph.value(s, nba_player.salary)
        if team == 'Brooklyn Nets' and rating > 90:
            full_name = str(graph.value(s, nba_player.full_name))
            dot_graph_federatedQuery.node(full_name)
            dot_graph_federatedQuery.node(team)
            dot_graph_federatedQuery.node(salary)
            dot_graph_federatedQuery.edge(full_name, team, label="fromTeam")
            dot_graph_federatedQuery.edge(full_name, salary, label="hasAnnualSalary")
            player_names.append(full_name)

TypeError: '>' not supported between instances of 'str' and 'int'

**Note**, I have a confusion in federated query which I do not really well understand about it.